# Part I. ETL Pipeline Process.

#### Import Python packages 

In [47]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [48]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [49]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Data Modelling Process. 

## CSV file titled <font color=red>event_datafile_new.csv</font>, is located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a setting Cluster and key space. 

In [50]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Establish connection and begin executing queries, need a session
session = cluster.connect()

# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except exception as e:
    print(e)

# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except exception as e:
    print(e)

### Modelling Apache Cassandra Database tables for the queries Provided.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [51]:
query = "DROP TABLE IF EXISTS Sparkify_Artist_Album_Info"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create table Sparkify_Artist_Album_Info to provide artist and song Information.
# table is designed with a Compound primary Key in which session id is the partition key 
# and item_in_session is the clustering key.
query = "CREATE TABLE IF NOT EXISTS Sparkify_Artist_Album_Info "
query = query + "(session_id int, item_in_Session int, artist_name text, song_title text, song_length float,\
                   PRIMARY KEY (session_id,item_in_Session))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

# Read and Fetch the data from the CSV file to load.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO Sparkify_Artist_Album_Info (session_id,item_in_Session,artist_name,song_title,song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

# SELECT to verify that the data have been inserted into the table.
query = "select artist_name,song_title,song_length from Sparkify_Artist_Album_Info where session_id = 338 and item_in_Session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [52]:
query = "DROP TABLE IF EXISTS Sparkify_artist_album_user_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create table Sparkify_artist_album_user_info to provide artist,song and user Information 
# Table is designed with a compound primary key in which user_id is the Partition Key
# and session id and Item_in_Session make the clustering key.
query = "CREATE TABLE IF NOT EXISTS Sparkify_artist_album_user_info"
query = query + "(user_id int,Session_id int, item_in_session int,artist_name text, song_title text,user_first_name text,user_last_name text \
                   ,PRIMARY KEY ((user_id,session_id),item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Read and Fetch data from the CSV file to load.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO Sparkify_artist_album_user_info(user_id,Session_id,item_in_session,artist_name,song_title,user_first_name,user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]),str(line[9]),str(line[1]),str(line[4])))  

# SELECT statement to verify the data was entered into the table.
query = "select artist_name,song_title,user_first_name,user_last_name from Sparkify_artist_album_user_info where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_title,row.user_first_name,row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

In [53]:
query = "DROP TABLE IF EXISTS Sparkify_album_user_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create table Sparkify_album_user_info to provide user Information for song 'All Hands Against His Own'.
# Table is designed with a Compound primary Key in which song title is the partition key 
# and user id is the clustering key.
query = "CREATE TABLE IF NOT EXISTS Sparkify_album_user_info"
query = query + "(song_title text,user_id int,user_first_name text,user_last_name text,PRIMARY KEY (song_title,user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

# Read and Fetch the data from the CSV file to load.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO Sparkify_album_user_info(song_title,user_id,user_first_name,user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
       
        session.execute(query, (str(line[9]),int(line[10]),str(line[1]),str(line[4])))

# SELECT statement to verify the data was entered into the table
query = "select song_title,user_id,user_first_name,user_last_name from Sparkify_album_user_info where song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name,row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [54]:
query = "DROP TABLE IF EXISTS Sparkify_Artist_Album_Info "
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS Sparkify_artist_album_user_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS Sparkify_album_user_info"
try:
    session.execute(query)
except Exception as e:
    print(e)    

### Close the session and cluster connection¶

In [55]:
session.shutdown()
cluster.shutdown()